In [6]:
import random
import sqlite3
import numpy as np
import pandas as pd

In [8]:
class TicTacToeGame():
    def __init__(self):
        self.reset()
    
    def play(self, player1, player2, gameCount = -1, display = False):
        play = 'y'
        if gameCount <= -1:
            play = input("Play? (y/n)")
        while play == 'y' and gameCount != 0:
            winner = self.game(player1, player2, display)
            
            if winner == 1:
                player1.win()
                player2.lose()
            elif winner == 2:
                player1.lose()
                player2.win()
            else:
                player1.tie()
                player2.tie()
                
            if gameCount <= -1:
                play = input("Play? (y/n)")
            else:
                gameCount -= 1
            
            
            
    def game(self, player1, player2, display):
        self.reset()
        while True:
            self.update(player1.move(self.state), 1)
            if display: self.display()
            finished = self.won()
            if finished: return finished
            
            self.update(player2.move(self.state), 2)
            if display: self.display()
            finished = self.won()
            if finished: return finished
       
    
    
    def update(self, move, player):
        self.state[move[0]][move[1]] = player
            
    def won(self):
        pass
    
    def reset(self):
        self.state = [[None, None, None],
                      [None, None, None],
                      [None, None, None]]
        
        
    def display(self):
        for row in self.state:
            print(row)
        print()
    

In [9]:
class Player():
    def __init__(self):
        pass
    
    def move(self, state):
        raise NotImplementedError
        
    def win():
        pass
    
    def lose():
        pass
    
    def tie():
        pass
    

    
class Human(Player):
    pass



class DataBot(Player):
    def __init__(self, player_number, ID):
        self.player_number = player_number
        self.conn, self.cur = self.initiateDB(ID)
        self.states = []
     
    
    
    def initiateDB(self, ID):
        conn = sqlite3.connect(str(ID) + '.db')
        cur = conn.cursor()
        try:
            cur.execute("""CREATE TABLE states
                         ([state] string PRIMARY KEY, [wins] integer, [loss] integer, [tie] integer)""")
        except:
            pass
        conn.commit()
        return conn, cur
        
        
        
    def move(self, state):
        all_moves = self.get_moves(state)
        data = self.get_data(all_moves)
        data = self.fill_data(all_moves, data)
        move = self.choose_move(data)
        self.states.append(move)
        return BoardHelper.difference(state, move[0])
    
    
    
    def get_moves(self, state):
        moves = []
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] == None:
                    copy = BoardHelper.copy(state)
                    copy[i][j] = self.player_number
                    moves.append(BoardHelper.board_to_string(copy))
        return moves
    
    
    
    def get_data(self, moves):
        self.cur.execute(f"""SELECT * FROM states WHERE state IN {str(moves).replace('[', '(').replace(']', ')')}""")
        return self.cur.fetchall()
    
    
    
    def fill_data(self, all_moves, data):
        moves_copy = all_moves.copy()
        full_data = []
        for move in data:
            moves_copy.remove(move[0])
            full_data.append(move)
        for move in moves_copy:
            full_data.append((move, 0, 0, 0))
        return full_data
        
        
        
    def choose_move(self, data):
        win = []
        unknown = []
        tie = []
        loss = []
        for move in data:
            if move[1] + move[2] + move[3] < 20:
                unknown.append(move)
            elif move[2] > 2 * (move[1] + move[3]): 
                loss.append(move)
            elif move[3] > 3 * (move[1] + move[2]):
                tie.append(move)
            elif move[1] > move[2]:
                win.append[move]
            else:
                unknown.append(move)
        if len(win) > 0:
            return random.choice(win)
        if len(unknown) > 0:
            return random.choice(unknown)
        if random.random() > 0.2 and len(tie) > 0:
            return random.choice(tie)
        return random.choice(loss)
        
        
        
    def win(self):
        pass
    
    
    
    def loss(self):
        pass
    
    
    
    def tie(self):
        pass
    
    
    
    
class BoardHelper():
    def board_to_string(board):
        player1 = ''
        player2 = ''
        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == 1:
                    player1 += f'{i}{j}'
                elif board[i][j] == 2:
                    player2 += f'{i}{j}'
        return player1 + '/' + player2
    
    
    
    def string_to_board(string):
        board = [[None, None, None],
                 [None, None, None],
                 [None, None, None]]
        split = string.split('/')
        for i in range(len(split)):
            for pair in [split[i][n: n+2] for n in range(0, len(split[i]), 2)]:
                board[int(pair[0])][int(pair[1])] = i + 1
        return board
    
    
    
    def difference(state, other):
        if type(state) == type('string'):
            state = BoardHelper.string_to_board(state)
        if type(other) == type('string'):
            other = BoardHelper.string_to_board(other)
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] != other[i][j]:
                    return (i, j)
        return None
    
    
    
    def copy(board):
        return [row.copy() for row in board]
                

In [11]:
ttt = TicTacToeGame()
ttt.play(DataBot(1, 1), DataBot(2, 2), 1, display = True)

[None, None, None]
[None, None, None]
[None, 1, None]

[None, 2, None]
[None, None, None]
[None, 1, None]

[None, 2, None]
[None, None, None]
[None, 1, 1]

[2, 2, None]
[None, None, None]
[None, 1, 1]

[2, 2, 1]
[None, None, None]
[None, 1, 1]

[2, 2, 1]
[None, 2, None]
[None, 1, 1]

[2, 2, 1]
[1, 2, None]
[None, 1, 1]

[2, 2, 1]
[1, 2, None]
[2, 1, 1]

[2, 2, 1]
[1, 2, 1]
[2, 1, 1]



IndexError: Cannot choose from an empty sequence

In [9]:
bot = DataBot(1,1)
states = bot.get_moves([[None,None,None],[None,None,None],[None,None,None]])
data = bot.get_data(states)

In [12]:
filled_data = bot.fill_data(states, data)

In [15]:
bot.choose_move(filled_data)

('12/', 0, 0, 0)

In [24]:
states

['00/', '01/', '02/', '10/', '11/', '12/', '20/', '21/', '22/']

In [8]:
import sqlite3

conn = sqlite3.connect('TestDB.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - CLIENTS
try:
    c.execute('''CREATE TABLE B
                 ([state] STRING,[win] integer, [loss] integer, [tie] integer)''')
except:
    pass
                 
conn.commit()


In [12]:
c.execute("""REPLACE INTO B VALUES ('b',2,2,3)""")

In [15]:
c.execute("""SELECT * FROM B WHERE state IN ('a', 'b')""")
c.fetchall()

[('a', 2, 2, 3), ('b', 2, 2, 3)]

In [16]:
conn.commit()

In [5]:
conn = sqlite3.connect('1.db')
c = conn.cursor()

In [6]:
c.execute("""REPLACE INTO states VALUES ('00/', 1, 1, 1)""")
conn.commit()

In [7]:
c.execute('SELECT * FROM states')
c.fetchall()

[('00/', 1, 1, 1)]